In [37]:
from pathlib import Path
import joblib
from typing import List

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

from pipeline import (
    DropHighNAPercentage,
    m_inst,
    OrdinalColumnMapper,
    DropColumns,
    DataframeOneHotEncoder,
)

pd.set_option("display.max_columns", None)

In [38]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"

DATA_PATH = DATA_DIR / 'indices_institucional_2005_2022.xlsx'
DATA_PATH.exists()

True

In [39]:
def show(df: pd.DataFrame, limit: int = 5) -> None:
    print(df.shape)
    display(df.head(limit))

In [40]:
def show_null_percentages(df: pd.DataFrame) -> None:
    nulls = df.isnull().sum() / df.shape[0]
    nulls = nulls.sort_values(ascending=False)
    display(nulls)

### Loading Data

In [41]:
# inmuebles = pd.read_excel(DATA_PATH, sheet_name="Inmuebles")
# labs = pd.read_excel(DATA_PATH, sheet_name="Laboratorios y Talleres")
# docentes = pd.read_excel(DATA_PATH, sheet_name="Docentes")

# extra_data = inmuebles, labs, docentes
# joblib.dump(extra_data, DATA_DIR / 'extra_data.pkl')

In [42]:
inmuebles, labs, docentes = joblib.load(DATA_DIR / 'extra_data.pkl')
df = joblib.load(DATA_DIR / 'processed_df.pkl')

### Preprocessing Data

In [43]:
def prepare_for_merge(df_to_merge: pd.DataFrame, year_col: str) -> pd.DataFrame:
    df_to_merge["Año"] = df_to_merge[year_col].astype(int)
    df_to_merge = df_to_merge.drop(columns=[year_col])
    df_to_merge = df_to_merge.rename(columns={"idSede": "Cód. Sede"})
    df_to_merge["Cód. Sede"] = df_to_merge["Cód. Sede"].astype(int)
    df_to_merge["Cód. Institución"] = df_to_merge["Cód. Institución"].astype(int)
    df_to_merge = df_to_merge.drop(columns=["Tipo Institución", "Cód. Institución"])

    return df_to_merge


inmuebles = prepare_for_merge(inmuebles, year_col="Año Información")
labs = prepare_for_merge(labs, year_col="Año Proceso")
docentes = prepare_for_merge(docentes, year_col="Año Proceso")

In [44]:
# pipes
high_na_perc_dropper = DropHighNAPercentage(na_threshold=0.3, exclude=[])

one_hot_encoder = DataframeOneHotEncoder(
    columns=["Nombre Institución", "Sede"], min_frequency=20
)
inputer = KNNImputer().set_output(transform="pandas")
# one_hot_encoder = OneHotEncoder(sparse_output=False, transform="pandas")

print("processing inmuebles...")
inmuebles = DropColumns(columns_to_drop=["idInstitucion"]).fit_transform(inmuebles)
inmuebles = high_na_perc_dropper.fit_transform(inmuebles)
inmuebles = one_hot_encoder.fit_transform(inmuebles)
inmuebles = inputer.fit_transform(inmuebles)

print("processing labs...")
labs = one_hot_encoder.fit_transform(labs)
labs = inputer.fit_transform(labs)

print("processing docentes...")
docentes = high_na_perc_dropper.fit_transform(docentes)
docentes = one_hot_encoder.fit_transform(docentes)
docentes = inputer.fit_transform(docentes)

processing inmuebles...
processing labs...
processing docentes...


In [45]:
show(inmuebles)
show(labs)
show(docentes)
show(df)

(5823, 122)


,Cód. Sede,N° Inmuebles,M2 Terreno,M2 Construido,M2 Salas,Nº Oficinas,Nº Salas,Año,Nombre Institución_C.F.T. ANDRÉS BELLO,Nombre Institución_C.F.T. CEDUC - UCN,Nombre Institución_C.F.T. DEL MEDIO AMBIENTE,Nombre Institución_C.F.T. ENAC,Nombre Institución_C.F.T. INACAP,Nombre Institución_C.F.T. LA ARAUCANA,Nombre Institución_C.F.T. LOS LAGOS,Nombre Institución_C.F.T. MASSACHUSETTS,Nombre Institución_C.F.T. PROANDES,Nombre Institución_C.F.T. PUCV,Nombre Institución_C.F.T. SALESIANOS DON BOSCO,Nombre Institución_C.F.T. SAN AGUSTÍN DE TALCA,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. TEODORO WICKEL,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DE ARTES Y COMUNICACIÓN ARCOS,Nombre Institución_I.P. DE CHILE,Nombre Institución_I.P. DE LOS LAGOS,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. DUOC UC,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. INACAP,Nombre Institución_I.P. INTERNACIONAL DE ARTES CULINARIAS Y SERVICIOS,Nombre Institución_I.P. IPEGE,Nombre Institución_I.P. LA ARAUCANA,Nombre Institución_I.P. LATINOAMERICANO DE COMERCIO EXTERIOR,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. PROVIDENCIA,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_I.P. VIRGINIO GÓMEZ,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. DE ACONCAGUA,Nombre Institución_U. DE ARTE Y CIENCIAS SOCIALES ARCIS,Nombre Institución_U. DE ATACAMA,Nombre Institución_U. DE LA FRONTERA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. TECNOLÓGICA DE CHILE INACAP,Nombre Institución_U. TECNOLÓGICA METROPOLITANA,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_infrequent_sklearn,Sede_Angol,Sede_Antofagasta,Sede_Arica,Sede_Buin,Sede_Calama,Sede_Cauquenes,Sede_Cañete,Sede_Chillán,Sede_Concepción,Sede_Constitución,Sede_Copiapó,Sede_Coquimbo,Sede_Coyhaique,Sede_Curicó,Sede_Iquique,Sede_La Ligua,Sede_La Reina,Sede_La Serena,Sede_Las Condes,Sede_Linares,Sede_Los Andes,Sede_Los Ángeles,Sede_Melipilla,Sede_Osorno,Sede_Ovalle,Sede_Providencia,Sede_Puerto Montt,Sede_Punta Arenas,Sede_Quillota,Sede_Rancagua,Sede_San Antonio,Sede_San Felipe,Sede_San Fernando,Sede_Santiago,Sede_Talca,Sede_Talcahuano,Sede_Temuco,Sede_Valdivia,Sede_Vallenar,Sede_Valparaíso,Sede_Vitacura,Sede_Viña del Mar,Sede_infrequent_sklearn
0,1001001.0,150.0,109043910.0,705759.0,50986.0,3500.0,708.0,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

(5567, 113)


,Cód. Sede,Nº Laboratorios,M2 Construido,Nº de PC para alumnos,N° Computadores con Internet,Año,Nombre Institución_C.F.T. ANDRÉS BELLO,Nombre Institución_C.F.T. CEDUC - UCN,Nombre Institución_C.F.T. ENAC,Nombre Institución_C.F.T. INACAP,Nombre Institución_C.F.T. LOS LAGOS,Nombre Institución_C.F.T. LOTA-ARAUCO,Nombre Institución_C.F.T. MASSACHUSETTS,Nombre Institución_C.F.T. PROANDES,Nombre Institución_C.F.T. PUCV,Nombre Institución_C.F.T. SALESIANOS DON BOSCO,Nombre Institución_C.F.T. SAN AGUSTÍN DE TALCA,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. TEODORO WICKEL,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DE ARTES Y COMUNICACIÓN ARCOS,Nombre Institución_I.P. DE CHILE,Nombre Institución_I.P. DE LOS LAGOS,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. DUOC UC,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. INACAP,Nombre Institución_I.P. INTERNACIONAL DE ARTES CULINARIAS Y SERVICIOS,Nombre Institución_I.P. IPEGE,Nombre Institución_I.P. LA ARAUCANA,Nombre Institución_I.P. LATINOAMERICANO DE COMERCIO EXTERIOR,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. PROVIDENCIA,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_I.P. VIRGINIO GÓMEZ,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. DE ACONCAGUA,Nombre Institución_U. DE ARTE Y CIENCIAS SOCIALES ARCIS,Nombre Institución_U. DE ATACAMA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. TECNOLÓGICA DE CHILE INACAP,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_infrequent_sklearn,Sede_Angol,Sede_Antofagasta,Sede_Arica,Sede_Calama,Sede_Cañete,Sede_Chillán,Sede_Concepción,Sede_Constitución,Sede_Copiapó,Sede_Coquimbo,Sede_Coyhaique,Sede_Curicó,Sede_Iquique,Sede_La Serena,Sede_Las Condes,Sede_Linares,Sede_Los Andes,Sede_Los Ángeles,Sede_Melipilla,Sede_Osorno,Sede_Ovalle,Sede_Providencia,Sede_Puerto Montt,Sede_Punta Arenas,Sede_Quillota,Sede_Rancagua,Sede_San Antonio,Sede_San Felipe,Sede_San Fernando,Sede_Santiago,Sede_Talca,Sede_Talcahuano,Sede_Temuco,Sede_Valdivia,Sede_Vallenar,Sede_Valparaíso,Sede_Vitacura,Sede_Viña del Mar,Sede_infrequent_sklearn
0,1001001.0,1091.0,64706.0,5325.0,5231.0,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001001.0,1103.0,65251.0,5190.0,5089.0,2021.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

(5845, 125)


,Cód. Sede,N°DocentesJornadaMedia,N°DocentesJornadaHora,N°DocentesJornadaCompleta,N°HorasJornadaCompleta,N°HorasJornadaMedia,N°HorasJornadaHora,N°HorasProfJornadaHora,N°HorasProfJornadaMedia,N°HorasProfJornadaCompleta,N°ProfesionalJornadaHora,N°ProfesionalJornadaMedia,N°ProfesionalJornadaCompleta,N°Docentes,N°Horas,N°DocentesHombres,N°DocentesMujeres,Año,Nombre Institución_C.F.T. ANDRÉS BELLO,Nombre Institución_C.F.T. CEDUC - UCN,Nombre Institución_C.F.T. DE TARAPACÁ,Nombre Institución_C.F.T. DIEGO PORTALES,Nombre Institución_C.F.T. INACAP,Nombre Institución_C.F.T. LOS LAGOS,Nombre Institución_C.F.T. PROANDES,Nombre Institución_C.F.T. PUCV,Nombre Institución_C.F.T. SALESIANOS DON BOSCO,Nombre Institución_C.F.T. SAN AGUSTÍN DE TALCA,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. TEODORO WICKEL,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DE ARTES Y COMUNICACIÓN ARCOS,Nombre Institución_I.P. DE CHILE,Nombre Institución_I.P. DE LOS LAGOS,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. DUOC UC,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. INACAP,Nombre Institución_I.P. INTERNACIONAL DE ARTES CULINARIAS Y SERVICIOS,Nombre Institución_I.P. IPEGE,Nombre Institución_I.P. LA ARAUCANA,Nombre Institución_I.P. LATINOAMERICANO DE COMERCIO EXTERIOR,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. PROVIDENCIA,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_I.P. VIRGINIO GÓMEZ,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. DE ACONCAGUA,Nombre Institución_U. DE CONCEPCIÓN,Nombre Institución_U. DE LA FRONTERA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. TECNOLÓGICA METROPOLITANA,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_infrequent_sklearn,Sede_Angol,Sede_Antofagasta,Sede_Arica,Sede_Bellavista,Sede_Calama,Sede_Cañete,Sede_Chillán,Sede_Concepción,Sede_Constitución,Sede_Copiapó,Sede_Coquimbo,Sede_Coyhaique,Sede_Curicó,Sede_Iquique,Sede_La Serena,Sede_Las Condes,Sede_Linares,Sede_Los Andes,Sede_Los Ángeles,Sede_Melipilla,Sede_Osorno,Sede_Ovalle,Sede_Providencia,Sede_Puerto Montt,Sede_Punta Arenas,Sede_Quillota,Sede_Rancagua,Sede_San Antonio,Sede_San Felipe,Sede_San Fernando,Sede_Santiago,Sede_Talca,Sede_Talcahuano,Sede_Temuco,Sede_Valdivia,Sede_Valparaíso,Sede_Villarrica,Sede_Vitacura,Sede_Viña del Mar,Sede_infrequent_sklearn
0,1001001.0,680.0,1584.0,1727.0,75316.0,15040.0,14201.0,4117.0,4299.0,8129.0,464.0,194.0,190.0,3991.0,104557.0,2440.0,1551.0,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0

(39726, 42)


,Fecha,Cód. Institución,Nombre Institución,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,Comuna donde se imparte la carrera o programa,Nombre Region,Cód. Carrera,Carrera Genérica,Nombre Programa,Horario,Tipo Programa,Area Conocimiento,Tipo Carrera,IngresoDirecto,Año Inicio Actividades,Nombre del Campus,Duración (en semestres),Cód. Sede,Título,Grado Académico,Promedio Puntaje (promedio matemáticas y lenguaje),Puntaje de corte (primer seleccionado),Puntaje de corte (promedio de la carrera),Puntaje de corte (último seleccionado),Nº Alumnos Ingreso Via PSU o PDT,Valor de matrícula,Valor de arancel,Valor del Título,Vacantes,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula Total,Año
0,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,Santiago,Región Metropolitana,7444,Administración pública y similares,Administración Pública,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,1954,Andrés Bello,11.0,1001001,Administrador Público,Licenciatura en Ciencias Políticas y Gubername...,702.0,919.0,758.0,708.0,129.0,177300.0,4861900.0,121000.0,130.0,68.0,82.0,150,302.0,392.0,694,2023
1,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,Santiago,Región Metropolitana,27288,"Contabilidad, Auditoría y similares",Contador Auditor,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,2013,Andrés Bello,10.0,1001001,Contador Auditor,Licenciado en Sistemas de Información y Contro...,766.0,885.0,750.0,704.0,84.0,177300.0,6137900.0,121000.0,70.0,54.0,45.0,99,191.0,205.0,396,2023
2,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,Santiago,Región Metropolitana,7420,Ingeniería Comercial,Ingeniería Comercial,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,1924,Andrés Bello,10.0,1001001,Ingeniero Comercial,Licenciatura en Ciencias Económicas o en Cienc...,863.0,963.0,838.0,803.0,419.0,177300.0,6844100.0,121000.0,380.0,353.0,234.0,587,1535.0,1126.0,2661,2023
3,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,Santiago,Región Metropolitana,27287,Ingeniería en Control de Gestión y similares,Ingeniería en Información y Control de Gestión,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,2013,Andrés Bello,10.0,1001001,Ingeniero en Información y Control de Gestión,Licenciado en Sistemas de Información y Contro...,771.0,881.0,752.0,701.0,172.0,177300.0,6137900.0,121000.0,140.0,101.0,86.0,187,395.0,358.0,753,2023
4,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,La Pintana,Región Metropolitana,7417,Ingeniería Agronómica,Ingeniería Agronómica,Diurno,Programa Regular,"Agricultura, Silvicultura, Pesca y Veterinaria",Profesional con Licenciatura,Ingreso Directo,1915,Sur,10.0,1001001,Ingeniero Agrónomo,Licenciado en Ciencias Agropecuarias,649.0,830.0,674.0,603.0,129.0,177300.0,5681600.0,121000.0,130.0,76.0,56.0,132,378.0,268.0,646,2023


In [46]:
print(f"Total number of columns: {inmuebles.shape[1] + labs.shape[1] + docentes.shape[1]}")

Total number of columns: 360


In [47]:
MERGE_COLS = [
    "Año",
    "Cód. Sede",
]

In [48]:
df_to_merge = docentes

X_values = df[MERGE_COLS].to_records(index=False).tolist()
Y_values = df_to_merge[MERGE_COLS].to_records(index=False).tolist()

X = set(X_values)
Y = set(Y_values)

In [49]:
len(X), len(Y), len(X & Y), len(X - Y), len(Y - X)

(2220, 5845, 1885, 335, 3960)

In [50]:
X_tuples = df[MERGE_COLS].apply(lambda x: tuple(x), axis=1)
notna_perc = X_tuples[X_tuples.isin(Y)].shape[0] / X_tuples.shape[0]
print(f"Percentage of not null values: {notna_perc * 100:.2f}%")

Percentage of not null values: 87.95%


In [56]:
def merge_dataframes(
    target_df: pd.DataFrame,
    df_to_merge: pd.DataFrame,
    merge_cols: List[str],
) -> pd.DataFrame:
    """
    Merge two dataframes on the specified columns,
    conserving the index of the target dataframe
    Args:
        target_df (pd.DataFrame): dataframe to merge with
        df_to_merge (pd.DataFrame): dataframe to merge
        merge_cols (List[str]): columns to merge on
    Returns:
        pd.DataFrame: merged dataframe
    """
    target_df = target_df.copy()
    
    target_df["index"] = target_df.index
    df_merged = target_df.merge(df_to_merge, on=merge_cols, how="inner")
    df_merged = df_merged.set_index("index")
    df_merged.index.name = None
    return df_merged

In [57]:
df_merged = merge_dataframes(df, inmuebles, MERGE_COLS)
df_merged = merge_dataframes(df_merged, labs, MERGE_COLS)
df_merged = merge_dataframes(df_merged, docentes, MERGE_COLS)

In [58]:
show(df_merged)

(30607, 396)


,Fecha,Cód. Institución,Nombre Institución,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,Comuna donde se imparte la carrera o programa,Nombre Region,Cód. Carrera,Carrera Genérica,Nombre Programa,Horario,Tipo Programa,Area Conocimiento,Tipo Carrera,IngresoDirecto,Año Inicio Actividades,Nombre del Campus,Duración (en semestres),Cód. Sede,Título,Grado Académico,Promedio Puntaje (promedio matemáticas y lenguaje),Puntaje de corte (primer seleccionado),Puntaje de corte (promedio de la carrera),Puntaje de corte (último seleccionado),Nº Alumnos Ingreso Via PSU o PDT,Valor de matrícula,Valor de arancel,Valor del Título,Vacantes,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula Total,Año,N° Inmuebles,M2 Terreno,M2 Construido_x,M2 Salas,Nº Oficinas,Nº Salas,Nombre Institución_C.F.T. ANDRÉS BELLO_x,Nombre Institución_C.F.T. CEDUC - UCN_x,Nombre Institución_C.F.T. DEL MEDIO AMBIENTE,Nombre Institución_C.F.T. ENAC_x,Nombre Institución_C.F.T. INACAP_x,Nombre Institución_C.F.T. LA ARAUCANA,Nombre Institución_C.F.T. LOS LAGOS_x,Nombre Institución_C.F.T. MASSACHUSETTS_x,Nombre Institución_C.F.T. PROANDES_x,Nombre Institución_C.F.T. PUCV_x,Nombre Institución_C.F.T. SALESIANOS DON BOSCO_x,Nombre Institución_C.F.T. SAN AGUSTÍN DE TALCA_x,Nombre Institución_C.F.T. SANTO TOMÁS_x,Nombre Institución_C.F.T. TEODORO WICKEL_x,Nombre Institución_I.P. AIEP_x,Nombre Institución_I.P. DE ARTES Y COMUNICACIÓN ARCOS_x,Nombre Institución_I.P. DE CHILE_x,Nombre Institución_I.P. DE LOS LAGOS_x,Nombre Institución_I.P. DEL VALLE CENTRAL_x,Nombre Institución_I.P. DIEGO PORTALES_x,Nombre Institución_I.P. DUOC UC_x,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA_x,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX_x,Nombre Institución_I.P. INACAP_x,Nombre Institución_I.P. INTERNACIONAL DE ARTES CULINARIAS Y SERVICIOS_x,Nombre Institución_I.P. IPEGE_x,Nombre Institución_I.P. LA ARAUCANA_x,Nombre Institución_I.P. LATINOAMERICANO DE COMERCIO EXTERIOR_x,Nombre Institución_I.P. LOS LEONES_x,Nombre Institución_I.P. PROVIDENCIA_x,Nombre Institución_I.P. SANTO TOMÁS_x,Nombre Institución_I.P. VIRGINIO GÓMEZ_x,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE_x,Nombre Institución_U. ADOLFO IBÁÑEZ_x,Nombre Institución_U. ANDRÉS BELLO_x,Nombre Institución_U. ARTURO PRAT_x,Nombre Institución_U. AUSTRAL DE CHILE_x,Nombre Institución_U. AUTÓNOMA DE CHILE_x,Nombre Institución_U. BOLIVARIANA_x,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN_x,Nombre Institución_U. CATÓLICA DEL MAULE_x,Nombre Institución_U. CATÓLICA DEL NORTE_x,Nombre Institución_U. CENTRAL DE CHILE_x,Nombre Institución_U. DE ACONCAGUA_x,Nombre Institución_U. DE ARTE Y CIENCIAS SOCIALES ARCIS_x,Nombre Institución_U. DE ATACAMA_x,Nombre Institución_U. DE LA FRONTERA_x,Nombre Institución_U. DE LA SERENA_x,Nombre Institución_U. DE LAS AMÉRICAS_x,Nombre Institución_U. DE LOS LAGOS_x,Nombre Institución_U. DE MAGALLANES_x,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN_x,Nombre Institución_U. DE TALCA_x,Nombre Institución_U. DE TARAPACÁ_x,Nombre Institución_U. DE VALPARAÍSO_x,Nombre Institución_U. DE VIÑA DEL MAR_x,Nombre Institución_U. DEL ALBA_x,Nombre Institución_U. DEL BÍO-BÍO_x,Nombre Institución_U. DEL DESARROLLO_x,Nombre Institución_U. DEL MAR_x,Nombre Institución_U. DEL PACÍFICO_x,Nombre Institución_U. GABRIELA MISTRAL_x,Nombre Institución_U. LA REPÚBLICA_x,Nombre Institución_U. MAYOR_x,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN_x,Nombre Institución_U. SAN SEBASTIÁN_x,Nombre Institución_U. SANTO TOMÁS_x,Nombre Institución_U. TECNOLÓGICA DE CHILE INACAP_x,Nombre Institución_U. TECNOLÓGICA METROPOLITANA_x,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA_x,Nombre Institución_infrequent_sklearn_x,Sede_Angol_x,Sede_Antofagasta_x,Sede_Arica_x,Sede_Buin,Sede_Calama_x,Sede_Cauquenes,Sede_Cañete_x,Sede_Chillán_x,Sede_Concepción_x,Sed

In [23]:
docentes

,Cód. Institución,Cód. Sede,Tipo Institución,N°DocentesJornadaMedia,N°DocentesJornadaHora,N°DocentesJornadaCompleta,N°HorasJornadaCompleta,N°HorasJornadaMedia,N°HorasJornadaHora,N°HorasProfJornadaHora,N°HorasProfJornadaMedia,N°HorasProfJornadaCompleta,N°ProfesionalJornadaHora,N°ProfesionalJornadaMedia,N°ProfesionalJornadaCompleta,N°Docentes,N°Horas,N°DocentesHombres,N°DocentesMujeres,Nombre Institución_C.F.T. ANDRÉS BELLO,Nombre Institución_C.F.T. CEDUC - UCN,Nombre Institución_C.F.T. DE TARAPACÁ,Nombre Institución_C.F.T. DIEGO PORTALES,Nombre Institución_C.F.T. INACAP,Nombre Institución_C.F.T. LOS LAGOS,Nombre Institución_C.F.T. PROANDES,Nombre Institución_C.F.T. PUCV,Nombre Institución_C.F.T. SALESIANOS DON BOSCO,Nombre Institución_C.F.T. SAN AGUSTÍN DE TALCA,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. TEODORO WICKEL,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DE ARTES Y COMUNICACIÓN ARCOS,Nombre Institución_I.P. DE CHILE,Nombre Institución_I.P. DE LOS LAGOS,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. DUOC UC,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. INACAP,Nombre Institución_I.P. INTERNACIONAL DE ARTES CULINARIAS Y SERVICIOS,Nombre Institución_I.P. IPEGE,Nombre Institución_I.P. LA ARAUCANA,Nombre Institución_I.P. LATINOAMERICANO DE COMERCIO EXTERIOR,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. PROVIDENCIA,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_I.P. VIRGINIO GÓMEZ,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. DE ACONCAGUA,Nombre Institución_U. DE CONCEPCIÓN,Nombre Institución_U. DE LA FRONTERA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. TECNOLÓGICA METROPOLITANA,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_infrequent_sklearn,Sede_Angol,Sede_Antofagasta,Sede_Arica,Sede_Bellavista,Sede_Calama,Sede_Cañete,Sede_Chillán,Sede_Concepción,Sede_Constitución,Sede_Copiapó,Sede_Coquimbo,Sede_Coyhaique,Sede_Curicó,Sede_Iquique,Sede_La Serena,Sede_Las Condes,Sede_Linares,Sede_Los Andes,Sede_Los Ángeles,Sede_Melipilla,Sede_Osorno,Sede_Ovalle,Sede_Providencia,Sede_Puerto Montt,Sede_Punta Arenas,Sede_Quillota,Sede_Rancagua,Sede_San Antonio,Sede_San Felipe,Sede_San Fernando,Sede_Santiago,Sede_Talca,Sede_Talcahuano,Sede_Temuco,Sede_Valdivia,Sede_Valparaíso,Sede_Villarrica,Sede_Vitacura,Sede_Viña del Mar,Sede_infrequent_sklearn,Año
0,1001.0,1001001,0.0,680.0,1584.0,1727.0,75316.0,15040.0,14201.0,4117.0,4299.0,8129.0,464.0,194.0,190.0,3991.0,104557.0,2440.0,1551.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0